<font color=red>__Important notice:   
place make sure to install the scikit-learn customized package before running this notebook to activated the ordinal models.  
Installation guidance are in the README file__</font>

In [62]:
https://www.mdpi.com/1099-4300/22/8/871

SyntaxError: invalid syntax (<ipython-input-62-13f572485711>, line 1)

# A Decision-Trees Based Approach for Ordinal Classification Problems

In [13]:
#Import packages  

from progressbar import ProgressBar
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import operator
import sklearn.metrics as  metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import scipy.stats as stats
from sklearn.ensemble import VotingClassifier
import itertools
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from scipy import stats
#from xgboost import XGBClassifier


In [14]:
# to ignore sklearn errors 
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## This function run our experiments:
Inputs:  

- list of models, each model is a tuple (index, model_name, model_object untrained)
- data sets as a dataframe 
- rnd and spp – hyper parameters to create kfold object 
  
Logic:  
- Initialize lists to save models scores both for model total score and per class 
- Run each model on the dataset and calculate all predefine metrics 
    - Initialize lists to save models scores both for model total score and per class per model 
    - Initialize kfold object, make sure each model been calculated on the same splits (5 folds)
    - Loop that run per kfold split:
        - Train the model
        - Predict on test set 
        - Calculate all metrics 
    - Save for each model family their counterpart
    - Check if compression should be made, if yes calculate p_value else put default values 
    - Average all fold results to one score. 
- Create 2 dataframes , one model level second class level and return them 

Output:
- 2 Dataframe with model score. First contain average score second with score std 
- 2 Dataframe with model score per class. First contain average score second with score std
 


In [34]:
#run each model
def calc_results(models,data,kfold,rnd = 1,spp = 20):
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    
    #Initialize lists to save models scores 
    pred_df = []
    pred__proba_df = []
    
    names = []

    model_lvl_accuracy_score_avg = []
    model_lvl_accuracy_score_std = []

    model_lvl_f1_weighted_score_avg = []
    model_lvl_f1_weighted_score_std = []

    model_lvl_recall_score_avg = []
    model_lvl_recall_score_std = []

    model_lvl_precision_score_avg =[]
    model_lvl_precision_score_std =[]

    model_lvl_MSE_score_avg = []
    model_lvl_MSE_score_std = []

    model_lvl_roc_auc_avg = []
    model_lvl_roc_auc_std = []

    model_lvl_kendall_tau = []
    model_lvl_kendall_p_value = []

    model_lvl_ttest_p_value_roc = []
    model_lvl_ttest_p_value_auc = []
    model_lvl_ttest_p_value_f1 = []
    model_lvl_ttest_p_value_recall = []
    model_lvl_ttest_p_value_precision = []
    model_lvl_ttest_p_value_MSE = []
    model_lvl_ttest_p_value_kendall = []
    
    ttest_champ_roc= []
    ttest_champ_auc= []
    ttest_champ_f1= []
    ttest_champ_recall= []
    ttest_champ_precision= []
    ttest_champ_MSE= []
    ttest_champ_kendall= []
    
    curr_champ = '989'
    
    #class
    class_lvl_MSE_avg = []
    class_lvl_MSE_std = []
    
    class_lvl_EV_avg = []
    class_lvl_EV_std = []
    
    class_lvl_roc_auc_avg = []
    class_lvl_roc_auc_std = []

    class_lvl_f1_score_avg = []
    class_lvl_f1_score_std = []

    class_lvl_recall_score_avg = []
    class_lvl_recall_score_std = []

    class_lvl_precision_score_avg = []
    class_lvl_precision_score_std = []
    
    
    print('total models : ', len(models))
    pbar = ProgressBar()
    
    # Loop that run each model on the dataset 
    for num,name,model in pbar(models):
    
        #metrics per model 
        accuracy_results = []
        f1_weighted_results = []
        recall_results = []
        precision_results =[]
        MSE = []
        roc_auc = []
        kendall_tau = []
        kendall_p_value = []
        
        #metric per class
        class_MSE_results = [] 
        class_EV_results = [] 
        class_roc_auc = []
        class_f1_results = []
        class_recall_results = []
        class_precision_results = [] 


        #Initialize kfold object
        kfold = StratifiedKFold(n_splits=spp,shuffle = True,random_state = rnd)
        for train_index, test_index in kfold.split(X,Y):

            # Fix for the ordinal binary model approach. 
            # This model can’t be retrained after trained once so we need to Initialize new model after each fold 
            #(code can be found : https://github.com/JOEYGLASSER/CLEVELAND-HEARTS-ANALYSIS)
            if name=='OrdinalModel':
                model =  OrdinalModel()   

            #split for train and test based on kfold index
            X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
            y_train, y_test = Y.loc[train_index], Y.loc[test_index]
            
            #train the model
            model.fit(X_train,y_train)
            
            #predict vector 
            pred = model.predict(X_test)
            
            #Calculate each metric per fold averaged by weighted, 
            #this option take into consideration the classes distribution 
            
            accuracy_results.append(metrics.accuracy_score(y_test,pred))
            f1_weighted_results.append(metrics.f1_score(y_test,pred,average = 'weighted'))
            recall_results.append(metrics.recall_score(y_test,pred,average = 'weighted'))
            precision_results.append(metrics.precision_score(y_test,pred,average = 'weighted'))
            MSE.append(metrics.mean_squared_error(y_test,pred))
            
            
            #Fix: when predict vector don’t contain all classes in the data set 
            #Add 1 line with the missing class, 
            #Assumption: since there are many samples and we calculate each metrics based on the class distribution , 
            #add 1 fake line with the missing class wont shift our scores 
            
            Y_uniqe = list(pd.unique(Y))
            Y_uniqe.sort()
            y_test_uniqe = list(pd.unique(y_test))
            pred_uniqe = list(pd.unique(pred))
            y_test_pd = pd.get_dummies(y_test)
            pred_pd = pd.get_dummies(pred)
            y_test_pd = y_test_pd.reset_index(drop = True)
            pred_pd = pred_pd.reset_index(drop = True)
            for label in Y_uniqe:
                if label not in pred_uniqe:
                    pred_pd[label] = 0

                if label not in y_test_uniqe:
            
                    y_test_pd[label] = 0 
                    extra_row = [0]*y_test_pd.shape[1]
                    extra_row[-1] = 1
                    y_test_pd.loc[len(y_test_pd)+2]= extra_row
                    pred_pd.loc[len(y_test_pd)-1]= [0]*pred_pd.shape[1]
                    y_test_pd = y_test_pd.reset_index(drop = True)
                    pred_pd = pred_pd.reset_index(drop = True)
                    
            cols = list(y_test_pd.columns)

            cols.sort()
            y_test_pd = y_test_pd[cols]
            pred_pd = pred_pd[cols]

 
            roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = 'weighted' , multi_class = 'ovr'))
            tau ,p_value =  stats.kendalltau(y_test,pred)
            kendall_tau.append(tau)
            kendall_p_value.append(p_value) 
            
            #Calculate each metric per fold per class          
            class_roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = None , multi_class = 'ovr'))
                  
            y_test_fix = pd.Series(y_test_pd.columns[np.where(y_test_pd!=0)[1]])
            pred_pd_fix = pd.Series(pred_pd.columns[np.where(pred_pd!=0)[1]])

            class_f1_results.append(metrics.f1_score(y_test_fix,pred_pd_fix,average = None))
            class_recall_results.append(metrics.recall_score(y_test_fix,pred_pd_fix,average = None))
            class_precision_results.append(metrics.precision_score(y_test_fix,pred_pd_fix,average = None))

            #Calculate MSE and EV per class
            pred_pd2 =pred
            y_test_pd2 =y_test.to_numpy(dtype='int64')
            MSE_res_fold = []
            EV_res_fold = []
            for label in Y_uniqe:
                pred_pd2_label = pred_pd2[np.argwhere(pred_pd2==label).T][0]
                y_test_pd2_label = y_test_pd2[np.argwhere(pred_pd2==label).T][0]
                if pred_pd2_label.shape[0] > 0: 
                    MSE_res_fold.append(metrics.mean_squared_error(pred_pd2_label,y_test_pd2_label))
                    #MSE_res_fold.append(metrics.mean_absolute_error(pred_pd2_label,y_test_pd2_label))
                    EV_res_fold.append(y_test_pd2_label.mean())
                else:
                    MSE_res_fold.append(0)
                    EV_res_fold.append(0)
                    
            class_MSE_results.append(MSE_res_fold)
            class_EV_results.append(EV_res_fold)
            
            #end of k-fold loop


        #Save for each model’s families their counterpart, that should be compared to   
        if len(num) == 1:
            ttest_champ_roc= roc_auc 
            ttest_champ_auc= accuracy_results
            ttest_champ_f1= f1_weighted_results
            ttest_champ_recall= recall_results
            ttest_champ_precision = precision_results
            ttest_champ_MSE = MSE
            ttest_champ_kendall   = kendall_tau
            curr_champ = num
            
        # Check if compression should be made
        # If yes: calculate p_value per metric 
        # If no : put default values    
        if curr_champ in num and len(num) != 1:

            model_lvl_ttest_p_value_roc.append(stats.ttest_rel(ttest_champ_roc,roc_auc)[1])
            model_lvl_ttest_p_value_auc.append(stats.ttest_rel(ttest_champ_auc,accuracy_results)[1])
            model_lvl_ttest_p_value_f1.append(stats.ttest_rel(ttest_champ_f1,f1_weighted_results)[1])
            model_lvl_ttest_p_value_recall.append(stats.ttest_rel(ttest_champ_recall,recall_results)[1])
            model_lvl_ttest_p_value_precision.append(stats.ttest_rel(ttest_champ_precision,precision_results)[1])
            model_lvl_ttest_p_value_MSE.append(stats.ttest_rel(ttest_champ_MSE,MSE)[1])     
            model_lvl_ttest_p_value_kendall.append(stats.ttest_rel(ttest_champ_kendall,kendall_tau)[1])         
          
        else:

            model_lvl_ttest_p_value_roc.append(9)
            model_lvl_ttest_p_value_auc.append(9)
            model_lvl_ttest_p_value_f1.append(9)
            model_lvl_ttest_p_value_recall.append(9)
            model_lvl_ttest_p_value_precision.append(9)
            model_lvl_ttest_p_value_MSE.append(9)  
            model_lvl_ttest_p_value_kendall.append(9) 

        #Average all fold results to one score. 
        pred_df.append([name,pred])
        
        model_lvl_accuracy_score_avg.append(np.average(accuracy_results))
        model_lvl_accuracy_score_std.append(np.std(accuracy_results))

        model_lvl_f1_weighted_score_avg.append(np.average(f1_weighted_results))
        model_lvl_f1_weighted_score_std.append(np.std(f1_weighted_results))

        model_lvl_recall_score_avg.append(np.average(recall_results))
        model_lvl_recall_score_std.append(np.std(recall_results))

        model_lvl_precision_score_avg.append(np.average(precision_results))
        model_lvl_precision_score_std.append(np.std(precision_results))

        model_lvl_MSE_score_avg.append(np.average(MSE))
        model_lvl_MSE_score_std.append(np.std(MSE))

        model_lvl_roc_auc_avg.append(np.average(roc_auc))
        model_lvl_roc_auc_std.append(np.std(roc_auc))
        
        model_lvl_kendall_tau.append(np.average(kendall_tau))
        model_lvl_kendall_p_value.append(np.average(kendall_p_value)) 

        class_lvl_MSE_avg.append(np.average(np.array(class_MSE_results) ,axis=0))
        class_lvl_MSE_std.append(np.std(np.array(class_MSE_results) ,axis=0))

        class_lvl_EV_avg.append(np.average(np.array(class_EV_results) ,axis=0))
        class_lvl_EV_std.append(np.std(np.array(class_EV_results) ,axis=0))
        
        class_lvl_roc_auc_avg.append(np.average(np.array(class_roc_auc) ,axis=0))
        class_lvl_roc_auc_std.append(np.std(np.array(class_roc_auc) ,axis=0))
        
        class_lvl_f1_score_avg.append(np.average(np.array(class_f1_results) ,axis=0))
        class_lvl_f1_score_std.append(np.std(np.array(class_f1_results) ,axis=0))

        class_lvl_recall_score_avg.append(np.average(np.array(class_recall_results) ,axis=0))
        class_lvl_recall_score_std.append(np.std(np.array(class_recall_results) ,axis=0))

        class_lvl_precision_score_avg.append(np.average(np.array(class_precision_results) ,axis=0))
        class_lvl_precision_score_std.append(np.std(np.array(class_precision_results) ,axis=0))
        
        names.append(name)
        indexs = list(range(1,len(names)+1))

    #Create 2 dataframes , one model level second class level and return them    
    data_avg = pd.DataFrame({'accuracy_score_avg':model_lvl_accuracy_score_avg ,
                            'f1_weighted_score_avg' :model_lvl_f1_weighted_score_avg ,
                            "recall_score_avg" :model_lvl_recall_score_avg ,
                            "precision_score_avg" :model_lvl_precision_score_avg ,
                            "MSE_score_avg" :model_lvl_MSE_score_avg,
                            'roc_auc_score_avg' : model_lvl_roc_auc_avg,
                            'kendall_tau' : model_lvl_kendall_tau , 
                            'ttest_p_value_roc' : model_lvl_ttest_p_value_roc ,               
                            'ttest_p_value_auc':model_lvl_ttest_p_value_auc,
                            'ttest_p_value_f1':model_lvl_ttest_p_value_f1,
                            'ttest_p_value_recall':model_lvl_ttest_p_value_recall,
                            'ttest_p_value_percision':model_lvl_ttest_p_value_precision,
                            'ttest_p_value_kendall'  :model_lvl_ttest_p_value_kendall , 
                            'ttest_p_value_MSE'   :model_lvl_ttest_p_value_MSE,
                             'indexs' : indexs
                } ,index=names)

    data_std = pd.DataFrame({'accuracy_score_std':model_lvl_accuracy_score_std ,
               'f1_weighted_score_std' :model_lvl_f1_weighted_score_std ,
               "recall_score_std" :model_lvl_recall_score_std ,
               "precision_score_std" :model_lvl_precision_score_std ,
               "MSE_score_std" :model_lvl_MSE_score_std,
                'roc_auc_score_std' : model_lvl_roc_auc_std,
                'kendall_p_value' : model_lvl_kendall_p_value
                            } ,index=names)

    #class level avg
    class_lvl_f1_score_avg = pd.DataFrame(class_lvl_f1_score_avg,index=names)
    class_lvl_f1_score_avg["metric"] = 'class_lvl_f1_score_avg'
    class_lvl_recall_score_avg = pd.DataFrame(class_lvl_recall_score_avg,index=names)
    class_lvl_recall_score_avg["metric"] = 'class_lvl_recall_score_avg'
    class_lvl_precision_score_avg = pd.DataFrame(class_lvl_precision_score_avg,index=names)
    class_lvl_precision_score_avg["metric"] = 'class_lvl_precision_score_avg'
    class_lvl_roc_auc_avg = pd.DataFrame(class_lvl_roc_auc_avg,index=names)
    class_lvl_roc_auc_avg["metric"] = 'class_lvl_roc_auc_avg'      
    class_lvl_MSE_avg = pd.DataFrame(class_lvl_MSE_avg,index=names)
    class_lvl_MSE_avg["metric"] = 'class_lvl_MSE_avg'  
    class_lvl_EV_avg = pd.DataFrame(class_lvl_EV_avg,index=names)
    class_lvl_EV_avg["metric"] = 'class_lvl_EV_avg'  

    data__class_avg = pd.concat([class_lvl_f1_score_avg,
                                 class_lvl_recall_score_avg,
                                 class_lvl_precision_score_avg,
                                 class_lvl_roc_auc_avg,
                                class_lvl_MSE_avg,
                                class_lvl_EV_avg])
    
    cols = data__class_avg.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data__class_avg = data__class_avg[cols]
    
    #class level std
    class_lvl_f1_score_std = pd.DataFrame(class_lvl_f1_score_std,index=names)
    class_lvl_f1_score_std["metric"] = 'class_lvl_f1_score_std'
    class_lvl_recall_score_std = pd.DataFrame(class_lvl_recall_score_std,index=names)
    class_lvl_recall_score_std["metric"] = 'class_lvl_recall_score_std'
    class_lvl_precision_score_std = pd.DataFrame(class_lvl_precision_score_std,index=names)
    class_lvl_precision_score_std["metric"] = 'class_lvl_precision_score_std'
    class_lvl_roc_auc_std = pd.DataFrame(class_lvl_roc_auc_std,index=names)
    class_lvl_roc_auc_std["metric"] = 'class_lvl_roc_auc_std'
    class_lvl_MSE_std = pd.DataFrame(class_lvl_MSE_std,index=names)
    class_lvl_MSE_std["metric"] = 'class_lvl_MSE_std'
    class_lvl_EV_std = pd.DataFrame(class_lvl_EV_std,index=names)
    class_lvl_EV_std["metric"] = 'class_lvl_EV_std'    
    
    data__class_std = pd.concat([class_lvl_f1_score_std,
                                 class_lvl_recall_score_std,
                                 class_lvl_precision_score_std,
                                 class_lvl_roc_auc_std,
                                 class_lvl_MSE_std,
                                 class_lvl_EV_std])
    
    cols = data__class_std.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data__class_std = data__class_std[cols]
    for i in range(3,8):
        if i not in list(pd.unique(Y)):
            data__class_avg[i] = np.nan
            data__class_std [i] = np.nan

    return data_avg.T,data_std.T,data__class_avg,data__class_std


## Optimization the normalization factor

The next sets of functions try to find the best normalization factor per data sets by running a grid search.  
Each function built for different algorithms families :
-	Decision trees
-	Random forests 
-	AdaBoost with normal boosting 
-	AdaBoost with ordinal boosting 


In [35]:
# Optimization the normalization factor for Decision trees
from numpy import arange

def WIGR_power_finder_DT(data,kfold,min_samples_leaf):
    print('WIGR_power_finder_DT:')
    powers1 = {}
    powers2 = {}
    powers3 = {}
    powers4 = {}
    powers5 = {}
    powers6 = {}
    
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    model_lvl_roc_auc_avg = []
    models_opt = []
    lst = list(arange(0.05, 1,0.2))+ list(arange(1, 3.5,0.5)) #list(range(0,11)) + list(arange(0.05, 1, 0.15))
    
    for i in lst:
        clf1 = DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)
        clf2 = DecisionTreeClassifier(criterion='WIGR_max',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)
        clf3 = DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)
        clf4 = DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)
        clf5 = DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)
        clf6 = DecisionTreeClassifier(criterion='entropy',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf)

      

        champ_roc = []
        for num,cl in [(6,(clf6,powers6)),(1,(clf1,powers1)),(2,(clf2,powers2)),(3,(clf3,powers3)),(4,(clf4,powers4)),(5,(clf5,powers5))]:
                     
            roc_auc = []
            
            for train_index, test_index in kfold.split(X,Y):
                
                x_train, x_test = X.loc[train_index,:], X.loc[test_index,:]
                y_train, y_test = Y.loc[train_index], Y.loc[test_index]
                
                cl[0].fit(x_train, y_train)


                pred = cl[0].predict(x_test)
                #handle roc_auc predict less classes than max
                Y_uniqe = list(pd.unique(Y))
                Y_uniqe.sort()
                y_test_uniqe = list(pd.unique(y_test))
                pred_uniqe = list(pd.unique(pred))
                y_test_pd = pd.get_dummies(y_test)
                pred_pd = pd.get_dummies(pred)
                y_test_pd = y_test_pd.reset_index(drop = True)
                pred_pd = pred_pd.reset_index(drop = True)

                for label in Y_uniqe:
                    if label not in pred_uniqe:
                        pred_pd[label] = 0


                    if label not in y_test_uniqe:
                        y_test_pd[label] = 0 
                        extra_row = [0]*y_test_pd.shape[1]
                        extra_row[-1] = 1
                        y_test_pd.loc[len(y_test_pd)+2]= extra_row
                        extra_row_pred = [0]*pred_pd.shape[1]
                        extra_row_pred[-1] = 1
                        pred_pd.loc[len(y_test_pd)-1]= extra_row_pred
                        y_test_pd = y_test_pd.reset_index(drop = True)
                        pred_pd = pred_pd.reset_index(drop = True)
                        
                roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = 'weighted' , multi_class = 'ovr'))


            if num == 6 : 
                champ_roc = roc_auc 
        
            if np.average(roc_auc) > np.average(champ_roc) and (stats.ttest_ind(champ_roc,roc_auc)[1] < 0.151 or stats.ttest_rel(champ_roc,roc_auc)[1]<0.151):
                print('jackpot!!!')
                
                cl[1][i] = np.average(roc_auc)
            else:
                cl[1][i] = np.average(roc_auc) - 0.2
        print(i)
        
    print  (list(zip([max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]],
            [powers1[max(powers1.items(), key=operator.itemgetter(1))[0]],
            powers2[max(powers2.items(), key=operator.itemgetter(1))[0]],
            powers3[max(powers3.items(), key=operator.itemgetter(1))[0]],
            powers4[max(powers4.items(), key=operator.itemgetter(1))[0]],
            powers5[max(powers5.items(), key=operator.itemgetter(1))[0]],
           powers6[ max(powers6.items(), key=operator.itemgetter(1))[0]]])))
    
    return  [max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]]

In [36]:
# Optimization the normalization factor for AdaBoost with normal boosting 
 
from numpy import arange

def WIGR_power_finder_ADA(data,kfold,min_samples_leaf):
    print('WIGR_power_finder_ADA:')
    powers1 = {}
    powers2 = {}
    powers3 = {}
    powers4 = {}
    powers5 = {}
    powers6 = {}
    
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    model_lvl_roc_auc_avg = []
    models_opt = []
    lst = list(arange(0.05, 1,0.2))+ list(arange(1, 3.5,0.5)) #list(range(0,11)) + list(arange(0.05, 1, 0.15))
    
    for i in lst:

        clf1 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))
        clf2 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_max',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))
        clf3 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))
        clf4 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))
        clf5 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))
        clf6 = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = 1))

      

        champ_roc = []
        for num,cl in [(6,(clf6,powers6)),(1,(clf1,powers1)),(2,(clf2,powers2)),(3,(clf3,powers3)),(4,(clf4,powers4)),(5,(clf5,powers5))]:
            
            roc_auc = []
            
            for train_index, test_index in kfold.split(X,Y):
                
                x_train, x_test = X.loc[train_index,:], X.loc[test_index,:]
                y_train, y_test = Y.loc[train_index], Y.loc[test_index]
                
                cl[0].fit(x_train, y_train)


                pred = cl[0].predict(x_test)
                #handle roc_auc predict less classes than max
                Y_uniqe = list(pd.unique(Y))
                Y_uniqe.sort()
                y_test_uniqe = list(pd.unique(y_test))
                pred_uniqe = list(pd.unique(pred))
                y_test_pd = pd.get_dummies(y_test)
                pred_pd = pd.get_dummies(pred)
                y_test_pd = y_test_pd.reset_index(drop = True)
                pred_pd = pred_pd.reset_index(drop = True)

                for label in Y_uniqe:
                    if label not in pred_uniqe:
                        pred_pd[label] = 0


                    if label not in y_test_uniqe:
                        y_test_pd[label] = 0 
                        extra_row = [0]*y_test_pd.shape[1]
                        extra_row[-1] = 1
                        y_test_pd.loc[len(y_test_pd)+2]= extra_row
                        extra_row_pred = [0]*pred_pd.shape[1]
                        extra_row_pred[-1] = 1
                        pred_pd.loc[len(y_test_pd)-1]= extra_row_pred
                        y_test_pd = y_test_pd.reset_index(drop = True)
                        pred_pd = pred_pd.reset_index(drop = True)
                        
                roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = 'weighted' , multi_class = 'ovr'))

         
            if num == 6 : 
                champ_roc = roc_auc 
        
            if np.average(roc_auc) > np.average(champ_roc) and (stats.ttest_ind(champ_roc,roc_auc)[1] < 0.151 or stats.ttest_rel(champ_roc,roc_auc)[1]<0.151):
                print('jackpot!!!')
                
                cl[1][i] = np.average(roc_auc)
            else:
                cl[1][i] = np.average(roc_auc) - 0.2
                
        print(i)
        
    print  (list(zip([max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]],
            [powers1[max(powers1.items(), key=operator.itemgetter(1))[0]],
            powers2[max(powers2.items(), key=operator.itemgetter(1))[0]],
            powers3[max(powers3.items(), key=operator.itemgetter(1))[0]],
            powers4[max(powers4.items(), key=operator.itemgetter(1))[0]],
            powers5[max(powers5.items(), key=operator.itemgetter(1))[0]],
           powers6[ max(powers6.items(), key=operator.itemgetter(1))[0]]])))
    
    return  [max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]]

In [37]:
# Optimization the normalization factor for AdaBoost with ordinal boosting 
from numpy import arange

def WIGR_power_finder_ADA_ordinal(data,kfold,min_samples_leaf,alg ='SAMME',Ordinal_problem =0,max_depth =1):
    print('WIGR_power_finder_ADA:' +alg + ' ' + str(Ordinal_problem) )
    powers1 = {}
    powers2 = {}
    powers3 = {}
    powers4 = {}
    powers5 = {}
    powers6 = {}
    
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    model_lvl_roc_auc_avg = []
    models_opt = []
    lst = list(arange(0.05, 1,0.2))+ list(arange(1, 3.5,0.5)) #list(range(0,11)) + list(arange(0.05, 1, 0.15))
    
    for i in lst:

        clf1 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = max_depth),Ordinal_problem=Ordinal_problem , algorithm = alg)
        clf2 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_max',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = max_depth),Ordinal_problem=Ordinal_problem , algorithm = alg)
        clf3 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = max_depth),Ordinal_problem=Ordinal_problem , algorithm = alg)
        clf4 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth =max_depth),Ordinal_problem=Ordinal_problem , algorithm = alg)
        clf5 = AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = max_depth),Ordinal_problem=Ordinal_problem , algorithm = alg)
        clf6 = AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,max_depth = max_depth), algorithm = alg)

        champ_roc = []
        for num,cl in [(6,(clf6,powers6)),(1,(clf1,powers1)),(2,(clf2,powers2)),(3,(clf3,powers3)),(4,(clf4,powers4)),(5,(clf5,powers5))]:
 
            roc_auc = []
            
            for train_index, test_index in kfold.split(X,Y):
                
                x_train, x_test = X.loc[train_index,:], X.loc[test_index,:]
                y_train, y_test = Y.loc[train_index], Y.loc[test_index]
                
                cl[0].fit(x_train, y_train)


                pred = cl[0].predict(x_test)
                #handle roc_auc predict less classes than max
                Y_uniqe = list(pd.unique(Y))
                Y_uniqe.sort()
                y_test_uniqe = list(pd.unique(y_test))
                pred_uniqe = list(pd.unique(pred))
                y_test_pd = pd.get_dummies(y_test)
                pred_pd = pd.get_dummies(pred)
                y_test_pd = y_test_pd.reset_index(drop = True)
                pred_pd = pred_pd.reset_index(drop = True)

                for label in Y_uniqe:
                    if label not in pred_uniqe:
                        pred_pd[label] = 0


                    if label not in y_test_uniqe:
                        y_test_pd[label] = 0 
                        extra_row = [0]*y_test_pd.shape[1]
                        extra_row[-1] = 1
                        y_test_pd.loc[len(y_test_pd)+2]= extra_row
                        extra_row_pred = [0]*pred_pd.shape[1]
                        extra_row_pred[-1] = 1
                        pred_pd.loc[len(y_test_pd)-1]= extra_row_pred
                        y_test_pd = y_test_pd.reset_index(drop = True)
                        pred_pd = pred_pd.reset_index(drop = True)
                        
                roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = 'weighted' , multi_class = 'ovr'))

         
            if num == 6 : 
                champ_roc = roc_auc 
        
            if np.average(roc_auc) > np.average(champ_roc) and (stats.ttest_ind(champ_roc,roc_auc)[1] < 0.151 or stats.ttest_rel(champ_roc,roc_auc)[1]<0.151):
                print('jackpot!!!')
                
                cl[1][i] = np.average(roc_auc)
            else:
                cl[1][i] = np.average(roc_auc) - 0.2
        print(i)
        
    print  (list(zip([max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]],
            [powers1[max(powers1.items(), key=operator.itemgetter(1))[0]],
            powers2[max(powers2.items(), key=operator.itemgetter(1))[0]],
            powers3[max(powers3.items(), key=operator.itemgetter(1))[0]],
            powers4[max(powers4.items(), key=operator.itemgetter(1))[0]],
            powers5[max(powers5.items(), key=operator.itemgetter(1))[0]],
           powers6[ max(powers6.items(), key=operator.itemgetter(1))[0]]])))
    
    return  [max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]]

In [38]:
# Optimization the normalization factor for Random forest

from numpy import arange

def WIGR_power_finder_RF(data,kfold,min_samples_leaf,n = 100):
    print('WIGR_power_finder_RF:')
    powers1 = {}
    powers2 = {}
    powers3 = {}
    powers4 = {}
    powers5 = {}
    powers6 = {}
    
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    model_lvl_roc_auc_avg = []
    models_opt = []
    lst = list(arange(0.05, 1,0.2))+ list(arange(1, 3.5,0.5)) #list(range(0,11)) + list(arange(0.05, 1, 0.15))
    
    for i in lst:
        clf1 = RandomForestClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators=  n)
        clf2 = RandomForestClassifier(criterion='WIGR_max',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators=  n)
        clf3 = RandomForestClassifier(criterion='WIGR_min',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators= n)
        clf4 = RandomForestClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators = n)
        clf5 = RandomForestClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators = n)
        clf6 = RandomForestClassifier(criterion='entropy',random_state=42,WIGR_power= i,min_samples_leaf=min_samples_leaf,n_estimators=  n)
        

        champ_roc = []
        for num,cl in [(6,(clf6,powers6)),(1,(clf1,powers1)),(2,(clf2,powers2)),(3,(clf3,powers3)),(4,(clf4,powers4)),(5,(clf5,powers5))]:
             
            roc_auc = []
            
            for train_index, test_index in kfold.split(X,Y):
                
                x_train, x_test = X.loc[train_index,:], X.loc[test_index,:]
                y_train, y_test = Y.loc[train_index], Y.loc[test_index]
                
                cl[0].fit(x_train, y_train)


                pred = cl[0].predict(x_test)
                #handle roc_auc predict less classes than max
                Y_uniqe = list(pd.unique(Y))
                Y_uniqe.sort()
                y_test_uniqe = list(pd.unique(y_test))
                pred_uniqe = list(pd.unique(pred))
                y_test_pd = pd.get_dummies(y_test)
                pred_pd = pd.get_dummies(pred)
                y_test_pd = y_test_pd.reset_index(drop = True)
                pred_pd = pred_pd.reset_index(drop = True)

                for label in Y_uniqe:
                    if label not in pred_uniqe:
                        pred_pd[label] = 0


                    if label not in y_test_uniqe:
                        y_test_pd[label] = 0 
                        extra_row = [0]*y_test_pd.shape[1]
                        extra_row[-1] = 1
                        y_test_pd.loc[len(y_test_pd)+2]= extra_row
                        extra_row_pred = [0]*pred_pd.shape[1]
                        extra_row_pred[-1] = 1
                        pred_pd.loc[len(y_test_pd)-1]= extra_row_pred
                        y_test_pd = y_test_pd.reset_index(drop = True)
                        pred_pd = pred_pd.reset_index(drop = True)
                        
                roc_auc.append(metrics.roc_auc_score(y_test_pd,pred_pd,average = 'weighted' , multi_class = 'ovr'))

         
            if num == 6 : 
                champ_roc = roc_auc 
        
            if np.average(roc_auc) > np.average(champ_roc) and (stats.ttest_ind(champ_roc,roc_auc)[1] < 0.151 or stats.ttest_rel(champ_roc,roc_auc)[1]<0.151):
                print('jackpot!!!')
                
                cl[1][i] = np.average(roc_auc)
            else:
                cl[1][i] = np.average(roc_auc) - 0.2
        print(i)
        
    print  (list(zip([max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]],
            [powers1[max(powers1.items(), key=operator.itemgetter(1))[0]],
            powers2[max(powers2.items(), key=operator.itemgetter(1))[0]],
            powers3[max(powers3.items(), key=operator.itemgetter(1))[0]],
            powers4[max(powers4.items(), key=operator.itemgetter(1))[0]],
            powers5[max(powers5.items(), key=operator.itemgetter(1))[0]],
           powers6[ max(powers6.items(), key=operator.itemgetter(1))[0]]])))
    
    return  [max(powers1.items(), key=operator.itemgetter(1))[0],
            max(powers2.items(), key=operator.itemgetter(1))[0],
            max(powers3.items(), key=operator.itemgetter(1))[0],
            max(powers4.items(), key=operator.itemgetter(1))[0],
            max(powers5.items(), key=operator.itemgetter(1))[0],
            max(powers6.items(), key=operator.itemgetter(1))[0]]

## Initialize and create all model for the experiment.
This function create all models object that will be run during our experiments 


Each tuple contain: (model_index, model_name, model object).    
For the counterpart index will be len== 1.   
For our ordinal model the index will be len==2 and the first digit equal to the counterpart 


In [61]:

def opt_models(data,kfold,min_samples_leaf):
    # Run the optimization function the normalization factor  
    power_wigr_DT = WIGR_power_finder_DT(data,kfold,min_samples_leaf)
    power_wigr_RF = WIGR_power_finder_RF(data,kfold,min_samples_leaf)
    power_wigr_ADA_SAMME = WIGR_power_finder_ADA_ordinal(data,kfold,min_samples_leaf,alg = 'SAMME')
    power_wigr_ADA_ordinal = WIGR_power_finder_ADA_ordinal(data,kfold,min_samples_leaf,Ordinal_problem = 1)

    models = []

    #Insert model to be compared and analyzed 
    models.append(('999',"Logistic Regression:",LogisticRegression(max_iter= 3)))
    models.append(('999',"Naive Bayes:",GaussianNB()))
    models.append(('999',"K-Nearest Neighbour:",KNeighborsClassifier(n_neighbors=3)))
    models.append(('999',"eXtreme Gradient Boost G:",XGBClassifier(min_child_weight  = min_samples_leaf )))
    models.append(('999',"GradientBoostingClassifier G:",GradientBoostingClassifier(min_samples_leaf=min_samples_leaf)))
    models.append(('999','OrdinalModel',OrdinalModel()))

    models.append(('1',"Decision Tree:",DecisionTreeClassifier(random_state = 42,criterion='entropy',min_samples_leaf=min_samples_leaf)))
    models.append(('10',"WIGR_mode Decision Tree",DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= power_wigr_DT[0],min_samples_leaf=min_samples_leaf)))
    models.append(('10',"WIGR_max Decision Tree",DecisionTreeClassifier(criterion='WIGR_max',random_state=42,WIGR_power= power_wigr_DT[1],min_samples_leaf=min_samples_leaf)))
    models.append(('10',"WIGR_min Decision Tree",DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= power_wigr_DT[2],min_samples_leaf=min_samples_leaf)))
    models.append(('10',"WIGR_EV Decision Tree",DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= power_wigr_DT[3],min_samples_leaf=min_samples_leaf)))
    models.append(('10',"WIGR_EV_fix Decision Tree",DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= power_wigr_DT[4],min_samples_leaf=min_samples_leaf)))

    models.append(('3',"Random Forest:",RandomForestClassifier(criterion='entropy',random_state=42,min_samples_leaf=min_samples_leaf)))
    models.append(('30',"WIGR_mode Random Forest",RandomForestClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= power_wigr_RF[0],min_samples_leaf=min_samples_leaf)))
    models.append(('30',"WIGR_max Random Forest",RandomForestClassifier(criterion='WIGR_max',random_state=42,WIGR_power= power_wigr_RF[1],min_samples_leaf=min_samples_leaf)))
    models.append(('30',"WIGR_min Random Forest",RandomForestClassifier(criterion='WIGR_min',random_state=42,WIGR_power= power_wigr_RF[2],min_samples_leaf=min_samples_leaf)))
    models.append(('30',"WIGR_EV Random Forest",RandomForestClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= power_wigr_RF[3],min_samples_leaf=min_samples_leaf)))
    models.append(('30',"WIGR_EV_fix Random Forest",RandomForestClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= power_wigr_RF[4],min_samples_leaf=min_samples_leaf)))

    models.append(('4',"AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=1),algorithm='SAMME')))
    models.append(('40',"WIGR_mode AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= power_wigr_ADA_SAMME[0]),algorithm='SAMME')))
    models.append(('40',"WIGR_max AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier (criterion='WIGR_max',random_state=42,WIGR_power=power_wigr_ADA_SAMME[1]),algorithm='SAMME')))
    models.append(('40',"WIGR_min AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= power_wigr_ADA_SAMME[2]),algorithm='SAMME')))
    models.append(('40',"WIGR_EV AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= power_wigr_ADA_SAMME[3]),algorithm='SAMME')))
    models.append(('40',"WIGR_EV_fix AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= power_wigr_ADA_SAMME[4]),algorithm='SAMME')))

    models.append(('5',"AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=1),algorithm='SAMME')))
    models.append(('50',"WIGR_mode AdaBoostClassifier Ordinal",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= power_wigr_ADA_ordinal[0]),Ordinal_problem=1)))
    models.append(('50',"WIGR_max AdaBoostClassifier Ordinal",AdaBoostClassifier(DecisionTreeClassifier (criterion='WIGR_max',random_state=42,WIGR_power=power_wigr_ADA_ordinal[1]),Ordinal_problem=1)))
    models.append(('50',"WIGR_min AdaBoostClassifier Ordinal",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_min',random_state=42,WIGR_power= power_wigr_ADA_ordinal[2]),Ordinal_problem=1)))
    models.append(('50',"WIGR_EV AdaBoostClassifier Ordinal",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV',random_state=42,WIGR_power= power_wigr_ADA_ordinal[3]),Ordinal_problem=1)))
    models.append(('50',"WIGR_EV_fix AdaBoostClassifier Ordinal",AdaBoostClassifier(DecisionTreeClassifier(criterion='WIGR_EV_fix',random_state=42,WIGR_power= pow.ar_wigr_ADA_ordinal[4]),Ordinal_problem=1)))

    models.append(('6',"AdaBoostClassifier SAMME",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=1),algorithm='SAMME')))
    models.append(('60',"AdaBoostClassifier boost Ordinal",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=42),Ordinal_problem=1)))

    models.append(('7',"AdaBoostClassifier SAMME2",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=1),algorithm='SAMME')))
    models.append(('70',"AdaBoostClassifier boost Ordinal2",AdaBoostClassifier(DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=1),Ordinal_problem=1)))
    
    
    #Ensemble experiment models   
#     permut = [
#         ("WIGR_mode Random Forest",RandomForestClassifier(criterion='WIGR_mode',random_state=42,WIGR_power= power_wigr_RF[0],min_samples_leaf=min_samples_leaf)),
#         ("WIGR_max Random Forest",RandomForestClassifier(criterion='WIGR_max',random_state=42,WIGR_power= power_wigr_RF[1],min_samples_leaf=min_samples_leaf)),
#         ("WIGR_min Random Forest",RandomForestClassifier(criterion='WIGR_min',random_state=42,WIGR_power= power_wigr_RF[2],min_samples_leaf=min_samples_leaf)),
#         ("eXtreme Gradient Boost:",XGBClassifier(min_child_weight  = min_samples_leaf )),
#         ("GradientBoostingClassifier:",GradientBoostingClassifier(min_samples_leaf=min_samples_leaf))
#     ]


#     models.append(('1',"Logistic Regression:",LogisticRegression(max_iter= 3)))
#     i=11
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))

#     models.append(('2',"eXtreme Gradient Boost:",XGBClassifier(min_child_weight  = min_samples_leaf,max_depth =1 )))
#     i=22
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))

#     models.append(('3',"GradientBoostingClassifier:",GradientBoostingClassifier(min_samples_leaf=min_samples_leaf,n_estimators =8)))
#     i=33
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))


#     models.append(('4','OrdinalModel',OrdinalModel()))
#     i=44
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))

#     models.append(('5',str(permut[0][0]),permut[0][1]))
#     i=55
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))
    
#     models.append(('6',str(permut[1][0]),permut[1][1]))
#     i=66
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))

#     models.append(('7',str(permut[2][0]),permut[2][1]))
#     i=77
#     vote = VotingClassifier(estimators=[permut[0]]+[permut[1]]+[permut[2]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[0][0]) + str(permut[1][0]) + str(permut[2][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[3]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[3][0]) +'hard',vote))
#     vote = VotingClassifier(estimators=[permut[2]]+[permut[1]]+[permut[4]],voting='hard')
#     models.append((str(i),str(i)+'_VotingClassifier_'+ str(permut[2][0]) + str(permut[1][0]) + str(permut[4][0]) +'hard',vote))


    print('Models appended...')
    return models

### This function run the experiment on all models per dataset and return the results 

In [56]:
def run_one_all(file,kfold,min_split):
    
    model_df = pd.DataFrame([])
    class_df = pd.DataFrame([])
    data_src = file
    print(data_src)
    data = pd.read_csv(file,header=None)
    rd =180
    sp = 5
    kfold = StratifiedKFold(n_splits=sp,shuffle = True,random_state = rd)
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1] #set label columns 
    print("number of class : ",len(pd.unique(Y)))
    models = opt_models (data,kfold,min_split)
    a,b,c,d= calc_results(models,data,kfold,rd,sp)
    
    for dfs in [a,b,c,d]:
        dfs['file'] = data_src.replace('.csv','')
        dfs['type'] = file.split('/')[1]
        if file.split('/')[1] != 'Orignial': 
            number_of_class = file.split('.')[0][-1]
            file_name = file.split('/')[-1][:-6]
        else:
            number_of_class = 3
            file_name = file.split('/')[-1][:-4]
        
        dfs['class'] = len(pd.unique(Y))
        dfs['min_split'] = min_split
        dfs['data'] = file_name
    model_df= pd.concat([model_df,a,b])
    class_df=pd.concat([class_df,c,d])
    print('----------------------------------------')
    model_df = model_df.reset_index()
    class_df = class_df.reset_index()
    return model_df,class_df,models

In [57]:
# Create kfold object 
kfold = StratifiedKFold(n_splits=20,shuffle = True,random_state = 69)

## This function run the experiment 

In [58]:
def run_all(path,kfold,min_split ):
    model_df = pd.DataFrame([])
    class_df = pd.DataFrame([])
    for path in paths:
    
        for data_src in os.listdir(path):
            for i in [10,15]:# range(10,11,10):
                
                if data_src=='.DS_Store':
                    continue
                model_df_curr,class_df_crr,models =  run_one_all(path + data_src,kfold,i)
                model_df= pd.concat([model_df,model_df_curr])
                class_df= pd.concat([class_df,class_df_crr])

    #            print('----------------------------------------')
            
    model_df = model_df.reset_index(drop = True)
    class_df = class_df.reset_index(drop = True)
    
    for x,m,n in models:
        model_df[m] = model_df[m].astype('object')
        
    return model_df,class_df

In [60]:
paths = [r'folder_name/'] # in which folder our data is kept 
results_per_model,results_per_class = run_all(paths,kfold,10)
results_per_model.to_excel('results_per_model.xlsx',index=False)
results_per_class.to_excel('results_per_class.xlsx',index=False)

100% |########################################################################|


3/pyrim_3.csv
number of class :  3
Models appended...
total models :  1
3/pyrim_3.csv
3/pyrim_3.csv
3/pyrim_3.csv
3/pyrim_3.csv
----------------------------------------
3/pyrim_3.csv
number of class :  3
Models appended...
total models :  1


100% |########################################################################|
100% |########################################################################|


3/pyrim_3.csv
3/pyrim_3.csv
3/pyrim_3.csv
3/pyrim_3.csv
----------------------------------------
3/stock_3.csv
number of class :  3
Models appended...
total models :  1
3/stock_3.csv
3/stock_3.csv
3/stock_3.csv
3/stock_3.csv
----------------------------------------
3/stock_3.csv
number of class :  3
Models appended...
total models :  1


100% |########################################################################|


3/stock_3.csv
3/stock_3.csv
3/stock_3.csv
3/stock_3.csv
----------------------------------------
